In [3]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
!head train.csv

3,0,240,663,-3,5,631,1,2,656,-2,3,636,-2,4,685,0,4,677,-2,1,666,-3,3,675,1,4
3,0,240,657,-3,5,629,1,2,652,-2,3,632,-2,4,681,0,4,676,-2,1,663,-3,3,671,1,4
3,0,240,652,-3,5,627,1,2,649,-2,3,628,-2,4,677,0,4,676,-2,1,659,-3,3,667,1,4
3,0,240,646,-3,5,625,1,2,646,-2,3,624,-2,4,673,0,4,675,-2,1,656,-3,3,663,1,4
3,0,240,641,-3,5,623,1,2,642,-2,3,621,-2,4,669,0,4,674,-2,1,652,-3,3,659,1,4
3,0,240,635,-3,5,621,1,2,639,-2,3,617,-2,4,665,0,4,673,-2,1,649,-3,3,655,1,4
3,0,240,629,-3,5,619,1,2,635,-2,3,613,-2,4,661,0,4,672,-2,1,645,-3,3,651,1,4
3,0,240,624,-3,5,617,1,2,632,-2,3,609,-2,4,657,0,4,671,-2,1,642,-3,3,647,1,4
3,0,240,618,-3,5,615,1,2,628,-2,3,606,-2,4,653,0,4,670,-2,1,638,-3,3,643,1,4
3,0,240,612,-3,5,613,1,2,625,-2,3,602,-2,4,649,0,4,670,-2,1,635,-3,3,639,1,4


In [27]:
features = ["f" + str(i) for i in range(0,26)]
label = ["action"]

headers = label + features

df = pd.read_csv('train.csv', 
                 sep = ',', 
                 header = None,
                 names = headers)
df

,action,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25
0,3,0,240,663,-3,5,631,1,2,656,...,4,677,-2,1,666,-3,3,675,1,4
1,3,0,240,657,-3,5,629,1,2,652,...,4,676,-2,1,663,-3,3,671,1,4
2,3,0,240,652,-3,5,627,1,2,649,...,4,676,-2,1,659,-3,3,667,1,4
3,3,0,240,646,-3,5,625,1,2,646,...,4,675,-2,1,656,-3,3,663,1,4
4,3,0,240,641,-3,5,623,1,2,642,...,4,674,-2,1,652,-3,3,659,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,3,0,112,551,1,5,628,-2,3,609,...,3,656,0,1,674,-2,2,680,1,1
756,3,0,112,546,1,5,625,-2,3,609,...,3,655,0,1,672,-2,2,679,1,1
757,3,0,112,541,1,5,621,-2,3,608,...,3,654,0,1,670,-2,2,678,1,1
758,3,0,112,537,1,5,618,-2,3,607,...,3,653,0,1,668,-2,2,678,1,1


In [6]:
y_train_all = df["action"]
X_train_all = df.drop(columns="action")

In [ ]:
# import pandas_profiling
# pandas_profiling.ProfileReport(df)

In [7]:
def evaluate_model(xtrain, ytrain, xtest, ytest, model):
    y_pred_test = model.predict(xtest)
    y_pred_train = model.predict(xtrain)
    test_accuracy = accuracy_score(ytest, y_pred_test)
    train_accuracy = accuracy_score(ytrain, y_pred_train)
    print('Training accuracy: {0:.2f}, evaluation accuracy: {1:.2f}'.format(train_accuracy, test_accuracy))


Basic logisting regresion

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all, test_size=0.25)
classifier = LogisticRegression(solver='lbfgs', max_iter=20000, multi_class="auto")
classifier.fit(X_train, y_train)


In [15]:
evaluate_model(X_train, y_train, X_test, y_test, classifier)

Training accuracy: 0.78, evaluation accuracy: 0.71


In [28]:
pickle.dump(mlp_model, open("ai_model_logit.pkl","wb"))

xgboost

In [16]:
y_train = df["action"]
X_train = df.drop(columns="action")

model = XGBClassifier().fit(X_train, y_train)

model.evals_result

<bound method XGBClassifier.evals_result of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)>

In [17]:
evaluate_model(X_train, y_train, X_test, y_test, model)

Training accuracy: 0.80, evaluation accuracy: 0.75


random forest model

In [18]:
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier(random_state=1, n_estimators=1000)
forest_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [20]:
evaluate_model(X_train, y_train, X_test, y_test, forest_model)

Training accuracy: 1.00, evaluation accuracy: 1.00


In [21]:
pickle.dump(forest_model, open("ai_model_forest.pkl","wb"))

In [23]:
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier(solver='adam', alpha=1e-5, 
                    hidden_layer_sizes=(128, 128, 128), random_state=1)

mlp_model.fit(X_train, y_train)   

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(128, 128, 128), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [29]:
evaluate_model(X_train, y_train, X_test, y_test, mlp_model)

Training accuracy: 0.65, evaluation accuracy: 0.68


In [24]:
pickle.dump(mlp_model, open("ai_model_mlp.pkl","wb"))

In [ ]:
# y_train = df[0]
# X_train = df.drop(columns=[0])
# y_train_matrix = y_train.values
# X_train_matrix = X_train.values

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns="action"), df["action"], 
                                                    test_size=0.2, random_state=7)

xgb_model = XGBClassifier(n_estimators=1000).fit(X_train, y_train,eval_set = [[X_test, y_test]])

[0]	validation_0-merror:0.269737
[1]	validation_0-merror:0.263158
[2]	validation_0-merror:0.263158
[3]	validation_0-merror:0.263158
[4]	validation_0-merror:0.263158
[5]	validation_0-merror:0.263158
[6]	validation_0-merror:0.263158
[7]	validation_0-merror:0.263158
[8]	validation_0-merror:0.263158
[9]	validation_0-merror:0.263158
[10]	validation_0-merror:0.263158
[11]	validation_0-merror:0.256579
[12]	validation_0-merror:0.256579
[13]	validation_0-merror:0.256579
[14]	validation_0-merror:0.256579
[15]	validation_0-merror:0.25
[16]	validation_0-merror:0.256579
[17]	validation_0-merror:0.256579
[18]	validation_0-merror:0.256579
[19]	validation_0-merror:0.243421
[20]	validation_0-merror:0.236842
[21]	validation_0-merror:0.236842
[22]	validation_0-merror:0.236842
[23]	validation_0-merror:0.236842
[24]	validation_0-merror:0.236842
[25]	validation_0-merror:0.236842
[26]	validation_0-merror:0.236842
[27]	validation_0-merror:0.236842
[28]	validation_0-merror:0.236842
[29]	validation_0-merror:0.2

[240]	validation_0-merror:0.388158
[241]	validation_0-merror:0.388158
[242]	validation_0-merror:0.388158
[243]	validation_0-merror:0.388158
[244]	validation_0-merror:0.394737
[245]	validation_0-merror:0.394737
[246]	validation_0-merror:0.394737
[247]	validation_0-merror:0.394737
[248]	validation_0-merror:0.401316
[249]	validation_0-merror:0.394737
[250]	validation_0-merror:0.388158
[251]	validation_0-merror:0.388158
[252]	validation_0-merror:0.394737
[253]	validation_0-merror:0.401316
[254]	validation_0-merror:0.401316
[255]	validation_0-merror:0.407895
[256]	validation_0-merror:0.407895
[257]	validation_0-merror:0.407895
[258]	validation_0-merror:0.407895
[259]	validation_0-merror:0.407895
[260]	validation_0-merror:0.407895
[261]	validation_0-merror:0.407895
[262]	validation_0-merror:0.407895
[263]	validation_0-merror:0.407895
[264]	validation_0-merror:0.407895
[265]	validation_0-merror:0.407895
[266]	validation_0-merror:0.407895
[267]	validation_0-merror:0.407895
[268]	validation_0-m

[475]	validation_0-merror:0.447368
[476]	validation_0-merror:0.440789
[477]	validation_0-merror:0.440789
[478]	validation_0-merror:0.440789
[479]	validation_0-merror:0.440789
[480]	validation_0-merror:0.440789
[481]	validation_0-merror:0.440789
[482]	validation_0-merror:0.440789
[483]	validation_0-merror:0.440789
[484]	validation_0-merror:0.440789
[485]	validation_0-merror:0.440789
[486]	validation_0-merror:0.440789
[487]	validation_0-merror:0.440789
[488]	validation_0-merror:0.447368
[489]	validation_0-merror:0.447368
[490]	validation_0-merror:0.447368
[491]	validation_0-merror:0.447368
[492]	validation_0-merror:0.447368
[493]	validation_0-merror:0.447368
[494]	validation_0-merror:0.447368
[495]	validation_0-merror:0.447368
[496]	validation_0-merror:0.447368
[497]	validation_0-merror:0.447368
[498]	validation_0-merror:0.447368
[499]	validation_0-merror:0.447368
[500]	validation_0-merror:0.447368
[501]	validation_0-merror:0.447368
[502]	validation_0-merror:0.447368
[503]	validation_0-m

[710]	validation_0-merror:0.460526
[711]	validation_0-merror:0.460526
[712]	validation_0-merror:0.460526
[713]	validation_0-merror:0.460526
[714]	validation_0-merror:0.460526
[715]	validation_0-merror:0.460526
[716]	validation_0-merror:0.460526
[717]	validation_0-merror:0.460526
[718]	validation_0-merror:0.460526
[719]	validation_0-merror:0.460526
[720]	validation_0-merror:0.460526
[721]	validation_0-merror:0.460526
[722]	validation_0-merror:0.460526
[723]	validation_0-merror:0.460526
[724]	validation_0-merror:0.460526
[725]	validation_0-merror:0.460526
[726]	validation_0-merror:0.460526
[727]	validation_0-merror:0.460526
[728]	validation_0-merror:0.460526
[729]	validation_0-merror:0.460526
[730]	validation_0-merror:0.467105
[731]	validation_0-merror:0.460526
[732]	validation_0-merror:0.467105
[733]	validation_0-merror:0.467105
[734]	validation_0-merror:0.467105
[735]	validation_0-merror:0.467105
[736]	validation_0-merror:0.467105
[737]	validation_0-merror:0.467105
[738]	validation_0-m

[945]	validation_0-merror:0.460526
[946]	validation_0-merror:0.460526
[947]	validation_0-merror:0.460526
[948]	validation_0-merror:0.460526
[949]	validation_0-merror:0.460526
[950]	validation_0-merror:0.460526
[951]	validation_0-merror:0.460526
[952]	validation_0-merror:0.460526
[953]	validation_0-merror:0.460526
[954]	validation_0-merror:0.460526
[955]	validation_0-merror:0.460526
[956]	validation_0-merror:0.460526
[957]	validation_0-merror:0.460526
[958]	validation_0-merror:0.460526
[959]	validation_0-merror:0.460526
[960]	validation_0-merror:0.460526
[961]	validation_0-merror:0.460526
[962]	validation_0-merror:0.460526
[963]	validation_0-merror:0.460526
[964]	validation_0-merror:0.460526
[965]	validation_0-merror:0.460526
[966]	validation_0-merror:0.460526
[967]	validation_0-merror:0.460526
[968]	validation_0-merror:0.460526
[969]	validation_0-merror:0.460526
[970]	validation_0-merror:0.460526
[971]	validation_0-merror:0.460526
[972]	validation_0-merror:0.460526
[973]	validation_0-m

In [26]:
pickle.dump(xgb_model, open("ai_model_xgb.pkl","wb"))

In [30]:
!ls

README.md           ai_model_forest.pkl img                 shmup-train.py
ai_model.ipynb      ai_model_mlp.pkl    old                 train.csv
ai_model.pkl        ai_model_xgb.pkl    shmup-play.py       train.log


In [ ]:
model_p = pickle.load(open("ai_model.pkl","rb"))

model_p.predict(np.array([0,240,685,-1,7,655,0,5,702,-1,3,647,2,4,715,-3,7,647,1,2,689,2,3,703,1,1]).reshape(1, -1))